### Simple implementation of linear regression

In [1]:
%matplotlib inline
import random
import torch
import numpy as np
from torch.utils import data

- synthetic data set

In [2]:
def synthetic_data(w, b, nums):
    # 生成一个形状为 (nums, len(w)) 的张量 X，
    # 每个元素都是从均值为 0 和标准差为 1 的正态分布中采样的随机数
    X = torch.normal(0, 1, (nums, len(w)))
    y = torch.matmul(X, w) + b

    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

real_w = torch.tensor([4.0, 8])
real_b = 13

features, labels = synthetic_data(real_w, real_b, 1000)

- Read data set

In [3]:
def load_array(data_arrays, batch_size, is_train = True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  # from torch.utils import data
    # shuffle = True 代表随机打乱数据
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# next(iter(data_iter))

- Definition model

In [4]:
from torch import nn
# nn是神经网络的缩写

 # 创建一个包含单个线性层（输入大小为2，输出大小为1）的顺序模型
net = nn.Sequential(nn.Linear(2, 1))

In [5]:
# 初始化随机参数

net[0].weight.data.normal_(0, 0.01)  # 用均值为0、标准差为0.01的正态分布初始化权重
net[0].bias.data.fill_(0)            # 将偏置初始化为0

tensor([0.])

- Loss Function

In [6]:
# loss = nn.MSELoss()
# loss = nn.SmoothL1Loss()
loss = nn.HuberLoss()

- Define optimization algorithm

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

- trainning

In [8]:
num_epochs = 7
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 10.725412
epoch 2, loss 8.448749
epoch 3, loss 6.270616
epoch 4, loss 4.186986
epoch 5, loss 2.216776
epoch 6, loss 0.465579
epoch 7, loss 0.002158


In [9]:
w = net[0].weight.data
b = net[0].bias.data
print(w)
print(b)
print('w的估计误差：', real_w - w.reshape(real_w.shape))
print('b的估计误差：', real_b - b)

tensor([[3.9666, 7.9495]])
tensor([12.9734])
w的估计误差： tensor([0.0334, 0.0505])
b的估计误差： tensor([0.0266])
